# Nest

Prepares data for the Nest page 

In [1]:
import os
import pandas as pd
from events import (
    load_event_data, filter_public_events, filter_by_project, prepare, save_files, count_wards, filter_signature_events, filter_community_events, filter_school_events, load_raw_data
)
import ticket_helpers


Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/nest/_data/events'
os.makedirs(OUT_DIR, exist_ok=True)

Read in event data and apply Airtable filters.

* Project_type is 'Produce (Signature)'
* Project_name is '10 - Melt'
* Event_type is 'Public Event -' and 'CLE - Community Engagement'

In [3]:
data = (
  load_raw_data()
    .pipe(filter_signature_events)
    .pipe(filter_by_project, '10 - Melt')
  )

Run the processing and save the files

In [4]:
data.pipe(filter_public_events).pipe(prepare).pipe(save_files, OUT_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
1093,1338.0,NaN,241,2023-09-09,Melt (Run),['Public Event - Performance'],10 - Melt: Melt (Run),1542.0,['LS26 8AL'],10 - Melt,Produce (Signature),['Season 3 - Dreaming'],2023-09-05,"ev_2809632, ev_2809633, ev_2809634, ev_28096...",['Garforth and Swillington'],Garforth and Swillington,E05013830,1338.0


## Ticket processing

Extract the ticket data

In [5]:
ticket_sales = data.pipe(filter_public_events).pipe(ticket_helpers.get_tickets_for_event)

By Ward

In [6]:
ticket_sales.pipe(ticket_helpers.summarise_by_ward).reset_index().to_csv(
    os.path.join(OUT_DIR, 'tickets_by_ward.csv'), index=False)


By Date

In [7]:
(
  ticket_sales
    .pipe(ticket_helpers.summarise_by_date)
    .resample('W-FRI').sum()
    .cumsum().to_csv(os.path.join(OUT_DIR, 'tickets_by_date.csv'))
)